In [282]:
import pandas as pd
import numpy as np
import json, requests
import xml.etree.ElementTree as ET
from matplotlib import pyplot as plt
from numpy import cov
import scipy.stats as p
from scipy.stats import chi2_contingency as chi
scenario = 0

# Single-Family Public Use Database JSON: Chi-Squared Tests

Data is from [The Exchange by Fannie Mae](https://theexchange.fanniemae.com), a data resource provided from Fannie, one of the Government Sponsered Enterprises (GSE) of the mortgage industry. The data has mortgage-level data and provides information on the borrower such as race/ethnicity, gender, and loan purpose. 

This analysis explores the data, makes transformations, and performs chi-squared tests on variables to see if there are relationships between them. 

In [283]:
f = open('Single-Family Public Use Database (PUDB).json', 'r')
data = pd.read_json(f)
f.close
Data = pd.DataFrame(data)data

,links,results,total
self,None,NaN,50768
results,None,NaN,50768
next,{'href': 'https://api.theexchange.fanniemae.co...,NaN,50768
previous,None,NaN,50768
embedded,NaN,"[{'enterprise': 'Freddie Mac', 'msaType': 'met...",50768


In [284]:
the_meat = data.iloc[4][1]
data = pd.DataFrame(the_meat)
data  

,enterprise,msaType,censusTractPctMinority,tractIncomeRatio,borrowerIncomeRatio,dateAcquiredVsDateOriginated,loanPurpose,federalGuarantee,sellerInstitutionType,borrowerRaceOriginEthnicity,coborrowerRaceOriginEthnicity,borrowerGender,coborrowerGender,occupancyCode,numberOfUnits,ownerOccupied,affordabilityCategory,reportingYear
0,Freddie Mac,metropolitan area,">=10, <30%",>120%,">50, <=80%",originated in same calendar year as acquired,purchase,"no Federal guarantee (i.e., conventional mortg...",Bank Insurance Fund (BIF)-insured depository i...,White,White,male,male,investment property (rental),1,no,Not available,2008
1,Freddie Mac,metropolitan area,">=10, <30%",>120%,>80%,originated prior to calendar year of acquisition,purchase,"no Federal guarantee (i.e., conventional mortg...",mortgage company,White,White,female,male,owner-occupied,1,yes,Not available,2008
2,Freddie Mac,metropolitan area,">=0, < 10%",">80, <=120%",>80%,originated in same calendar year as acquired,refinancing,"no Federal guarantee (i.e., conventional mortg...",mortgage company,White,White,male,female,owner-occupied,1,yes,Not available,2008
3,Freddie Mac,metropolitan area,">=10, <30%",>120%,>80%,originated in same calendar year as acquired,purchase,"no Federal guarantee (i.e., conventional mortg...",Savings Association Insurance Fund (SAIF)-insu...,not available/not applicable,not available/not applicable,male,no co-borrower,owner-occupied,1,yes,Not available,2008
4,Freddie Mac,metropolitan area,">=10, <30%",>120%,>80%,originated prior to calendar year of acquisition,refinancing,"no Federal guarantee (i.e., conventional mortg...",Bank Insurance Fund (BIF)-insured depository i...,not available/not applicable,not available/not applicable,male,female,owner-occupied,1,yes,Not available,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Freddie Mac,non-metropolitan area,">=30, <=100%",">80, <=120%",>80%,originated in same calendar year as acquired,refinancing,"no Federal guarantee (i.e., conventional mortg...",Savings Association Insurance Fund (SAIF)-insu...,White,White,male,female,owner-occupied,1,yes,Not available,2008
996,Freddie Mac,metropolitan area,">=10, <30%",>120%,not applicable,originated prior to calendar year of acquisition,refinancing,"no Federal guarantee (i.e., conventional mortg...",Bank Insurance Fund (BIF)-insured depository i...,not available/not applicable,not available/not applicable,missing,missing,owner-occupied,1,yes,Not available,2008
997,Freddie Mac,metropolitan area,">=0, < 10%",">80, <=120%",>80%,originated in same calendar year as acquired,refinancing,"no Federal guarantee (i.e., conventional mortg...",mortgage company,White,White,male,female,owner-occupied,1,yes,Not available,2008
998,Freddie Mac,metropolitan area,">=0, < 10%",">0, <=80%",>80%,originated in same calendar year as acquired,refinancing,"no Federal guarantee (i.e., conventional mortg...",Bank Insurance Fund (BIF)-insured depository i...,White,White,male,male,owner-occupied,1,yes,Not available,2008


## Explore Data and Distributions: 

Explore fields and determine percentages for each value of gender, metro/nonmetro, race/ethnicity, loan purpose, and income ratio.

In [285]:
counter = {'female':0, 'male':0, 'unknown':0}

for entry in data['borrowerGender']:
    if entry == 'female':
        counter['female'] = counter['female'] + 1
    elif entry == 'male':
        counter['male'] = counter['male'] + 1
    else:
        counter['unknown'] = counter['unknown'] + 1

counter

perc_female = 'Percent Female (of known): ' + str(round((counter['female']*100)/(counter['female']+counter['male']),2))+'%'
perc_male = 'Percent Male (of known): ' + str(round((counter['male']*100)/(counter['female']+counter['male']),2))+'%'
List = [perc_female, perc_male]
List

['Percent Female (of known): 30.81%', 'Percent Male (of known): 69.19%']

In [286]:
counter = {'metropolitan area':0, 'non-metropolitan area':0}

for entry in data['msaType']:
    if entry == 'metropolitan area':
        counter['metropolitan area'] = counter['metropolitan area'] + 1
    elif entry == 'non-metropolitan area':
        counter['non-metropolitan area'] = counter['non-metropolitan area'] + 1
    else: print('other value')

counter

perc_metro = 'Percent Metro: ' + str(round((counter['metropolitan area']*100)/(counter['metropolitan area']+counter['non-metropolitan area']),2))+'%'
perc_nonmetro = 'Percent Non-Metro: ' + str(round((counter['non-metropolitan area']*100)/(counter['metropolitan area']+counter['non-metropolitan area']),2))+'%'
List = [perc_metro, perc_nonmetro]
List

['Percent Metro: 88.3%', 'Percent Non-Metro: 11.7%']

In [287]:
counter = {'purchase':0, 'refinancing':0}

for entry in data['loanPurpose']:
    if entry == 'purchase':
        counter['purchase'] = counter['purchase'] + 1
    elif entry == 'refinancing':
        counter['refinancing'] = counter['refinancing'] + 1
    else: print('other value')

counter

perc_purchase = 'Percent Purchase: ' + str(round((counter['purchase']*100)/(counter['purchase']+counter['refinancing']),2))+'%'
perc_refi = 'Percent Refi: ' + str(round((counter['refinancing']*100)/(counter['purchase']+counter['refinancing']),2))+'%'
List = [perc_purchase, perc_refi]
List

['Percent Purchase: 37.3%', 'Percent Refi: 62.7%']

In [288]:
data['borrowerIncomeRatio'].unique()

#Income Ratio Counts
income_ratio_counts = data.groupby(['borrowerIncomeRatio']).count().iloc[:,0].to_dict()
income_ratio_counts

{'>50, <=80%': 247, '>80%': 539, '>= 0, <=50%': 143, 'not applicable': 71}

In [374]:
seller_institution_types = data.groupby(['sellerInstitutionType']).count().iloc[:,0].to_dict()
seller_institution_types

{'Bank Insurance Fund (BIF)-insured depository institution': 550,
 'Credit union insured by the National Credit Union Administration (NCUA)': 15,
 'Savings Association Insurance Fund (SAIF)-insured depository institution': 156,
 'mortgage company': 279}

In [289]:
race_eth = data.groupby(['borrowerRaceOriginEthnicity']).count().iloc[:,0].to_dict()
race_eth

{'American Indian or Alaska Native': 3,
 'Asian': 53,
 'Black or African American': 43,
 'Hispanic or Latino': 72,
 'Native Hawaiian or other Pacific Islander': 1,
 'Two or more races': 8,
 'White': 624,
 'not available/not applicable': 196}

## Recoding race/ethnicity:

We must recode the race/ethnicity column since many field values have very small sample sizes. We will recode any of the values with small sample sizes into an "Other Category", so the small sample sizes don't interfere with our Chi-Squared tests. Then, we will filter out the `Other` and `'not available/not applicable` values and only include the known values with large enough sample sizes. 

In [290]:
def borrower_race_eth(entry):
    if entry == 'American Indian or Alaska Native':
        v = 'Other'
    elif entry == 'Native Hawaiian or other Pacific Islander':
        v ='Other'
    elif entry == 'Two or more races':
        v = 'Other'
    else:
        v = entry
    return v

data['borrowerRaceOriginEthnicity'] = data['borrowerRaceOriginEthnicity'].apply(borrower_race_eth)

race_eth = data.groupby(['borrowerRaceOriginEthnicity']).count().iloc[:,0].to_dict()
race_eth

filter_out = ['Other', 'not available/not applicable']

data_race_eth = data[~data['borrowerRaceOriginEthnicity'].isin(filter_out)]
data_race_eth

,enterprise,msaType,censusTractPctMinority,tractIncomeRatio,borrowerIncomeRatio,dateAcquiredVsDateOriginated,loanPurpose,federalGuarantee,sellerInstitutionType,borrowerRaceOriginEthnicity,coborrowerRaceOriginEthnicity,borrowerGender,coborrowerGender,occupancyCode,numberOfUnits,ownerOccupied,affordabilityCategory,reportingYear
0,Freddie Mac,metropolitan area,">=10, <30%",>120%,">50, <=80%",originated in same calendar year as acquired,purchase,"no Federal guarantee (i.e., conventional mortg...",Bank Insurance Fund (BIF)-insured depository i...,White,White,male,male,investment property (rental),1,no,Not available,2008
1,Freddie Mac,metropolitan area,">=10, <30%",>120%,>80%,originated prior to calendar year of acquisition,purchase,"no Federal guarantee (i.e., conventional mortg...",mortgage company,White,White,female,male,owner-occupied,1,yes,Not available,2008
2,Freddie Mac,metropolitan area,">=0, < 10%",">80, <=120%",>80%,originated in same calendar year as acquired,refinancing,"no Federal guarantee (i.e., conventional mortg...",mortgage company,White,White,male,female,owner-occupied,1,yes,Not available,2008
5,Freddie Mac,metropolitan area,">=0, < 10%",">80, <=120%",">50, <=80%",originated prior to calendar year of acquisition,purchase,"no Federal guarantee (i.e., conventional mortg...",Bank Insurance Fund (BIF)-insured depository i...,White,White,male,male,investment property (rental),2,no,Not available,2008
6,Freddie Mac,metropolitan area,">=0, < 10%",">80, <=120%",>80%,originated in same calendar year as acquired,purchase,"no Federal guarantee (i.e., conventional mortg...",mortgage company,White,White,male,female,owner-occupied,1,yes,Not available,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,Freddie Mac,metropolitan area,">=30, <=100%",>120%,>80%,originated in same calendar year as acquired,refinancing,"no Federal guarantee (i.e., conventional mortg...",mortgage company,White,White,male,female,owner-occupied,1,yes,Not available,2008
995,Freddie Mac,non-metropolitan area,">=30, <=100%",">80, <=120%",>80%,originated in same calendar year as acquired,refinancing,"no Federal guarantee (i.e., conventional mortg...",Savings Association Insurance Fund (SAIF)-insu...,White,White,male,female,owner-occupied,1,yes,Not available,2008
997,Freddie Mac,metropolitan area,">=0, < 10%",">80, <=120%",>80%,originated in same calendar year as acquired,refinancing,"no Federal guarantee (i.e., conventional mortg...",mortgage company,White,White,male,female,owner-occupied,1,yes,Not available,2008
998,Freddie Mac,metropolitan area,">=0, < 10%",">0, <=80%",>80%,originated in same calendar year as acquired,refinancing,"no Federal guarantee (i.e., conventional mortg...",Bank Insurance Fund (BIF)-insured depository i...,White,White,male,male,owner-occupied,1,yes,Not available,2008


### New race values and counts:

In [291]:
race_eth = sorted(race_eth.items(), key = lambda x : x[1], reverse = True)
race_eth

[('White', 624),
 ('not available/not applicable', 196),
 ('Hispanic or Latino', 72),
 ('Asian', 53),
 ('Black or African American', 43),
 ('Other', 12)]

### Cross-tabulation:

Let's view the cross-tabulation of  `borrowerRaceOriginEthnicity` and `msaType` and then conduct a Chi-Squared test:

In [292]:
#NOTE: The results would be more interesting and accurate if there were more non-Metro observations
crosstab_race_metro = pd.crosstab(index = data['borrowerRaceOriginEthnicity'], columns = data['msaType'])
crosstab_race_metro

msaType,metropolitan area,non-metropolitan area
borrowerRaceOriginEthnicity,,
Asian,52,1
Black or African American,43,0
Hispanic or Latino,67,5
Other,12,0
White,530,94
not available/not applicable,179,17


In [293]:
chi_sq_result = p.chi2_contingency(crosstab_race_metro)
chi_sq_result
#returns ChiSquared test statistic, p value, degrees of freedom, and
#The expected frequencies, based on the marginal sums of the table

(22.377446282828558, 0.00044378112244336385, 5, array([[ 46.799,   6.201],
        [ 37.969,   5.031],
        [ 63.576,   8.424],
        [ 10.596,   1.404],
        [550.992,  73.008],
        [173.068,  22.932]]))

### Generalization and Function: 
Let's generalize the Chi-Squared test into a function `chi_sq_test` that takes 2 columns and a probability. The function will produce the results of the test while comparing the `alpha value` to the `p value` & the `chi-squared statistic` to the `critical value` under the hood.

In [396]:
def chi_sq_test(col1, col2, prob):
    
    crosstab_col1_col2 = pd.crosstab(index = col1, columns = col2)
    chi_sq_result = p.chi2_contingency(crosstab_col1_col2)
    
    chi_sq_stat = chi_sq_result[0]
    p_val = chi_sq_result[1]
    doff = chi_sq_result[2]
    expected = chi_sq_result[3]
    
    alpha = 1-prob
    critical_value = p.chi2.ppf(alpha, df=doff)
    
    
    if chi_sq_stat > critical_value:
        chi_stat_result = 'Chi-Squared Statisitc is > Critical value'
    else:
        chi_stat_result = 'Chi-Squared Statisitc is <= Critical value'
        
    if p_val <= alpha:
        p_result = 'P Value Result:' + str(p_val) + ', Reject the H0, Variables are dependent'
    else:
        p_result = 'P Value Result:' + str(p_val) + ', Fail to reject the H0, Variables are independent'

    return list([chi_stat_result, p_result])

### Relationship Analysis: Borrower Race/Ethnicity and Loan Purpose

Below explore the cross-tabulation, chi-squared output, and uses the new chi_sq_result function to compute a result. On the 95% significant level, we fail to reject the null hypothesis (i.e. H0 = variables are independent & have no relationship) and conclude that it is very probable that the variables are independent. 

In [402]:
#corss tabulation
crosstab_col1_col2 = pd.crosstab(index = data_race_eth['borrowerRaceOriginEthnicity'], columns = data_race_eth['loanPurpose'],normalize = 'index')
chi_sq_result = p.chi2_contingency(crosstab_col1_col2)

crosstab_col1_col2

loanPurpose,purchase,refinancing
borrowerRaceOriginEthnicity,,
Asian,0.490566,0.509434
Black or African American,0.441860,0.558140
Hispanic or Latino,0.444444,0.555556
White,0.394231,0.605769


In [403]:
#returns ChiSquared test statistic, p value, degrees of freedom, and expected frequencies
chi_sq_result

(0.0188231532566813, 0.9993170227210673, 3, array([[0.44277543, 0.55722457],
        [0.44277543, 0.55722457],
        [0.44277543, 0.55722457],
        [0.44277543, 0.55722457]]))

In [404]:
chi_sq_test(data_race_eth['borrowerRaceOriginEthnicity'], data_race_eth['loanPurpose'], .95)

['Chi-Squared Statisitc is > Critical value',
 'P Value Result:0.4599553359430485, Fail to reject the H0, Variables are independent']

### loanPurpose and sellerInstitutionType
On the 95% significant level, we reject the null hypothesis (i.e. H0 = variables are independent & have no relationship) and conclude that it is very probable that the variables are dependent and have a relationship.

In [405]:
chi_sq_test(data_race_eth['loanPurpose'], data_race_eth['sellerInstitutionType'], .95)

['Chi-Squared Statisitc is > Critical value',
 'P Value Result:0.006534542504333175, Reject the H0, Variables are dependent']

#### Export data frame as excel

In [356]:
#Save as excel file
data.to_excel(r'/Users/tushar/Desktop/data.xlsx')